In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/Users/ansintsova/git_repos/mBARq/tests/expected_outcomes/mapping/map_annotate.csv', index_col=0)

In [ ]:
df.head()

In [ ]:
pps = df[['sseqid', 'sstart', 'sstrand', 'barcode', 'cnt', 'total_count', 'multimap']]
        # Find barcodes mapped to the same position
collisions = (df.groupby(['sseqid', 'sstart']).barcode.nunique()
                      .reset_index()
                      .rename({'barcode': 'num_uniq_bc'}, axis=1))
        # Get all the unique barcodes mapped to unique positions
final_bcs = list(collisions[collisions.num_uniq_bc == 1].merge(pps, on=['sseqid', 'sstart']).barcode.values)
        # Collisions are only positions with more than one bc mapped
collisions = collisions[collisions.num_uniq_bc > 1]
collisions = collisions.merge(pps, on=['sseqid', 'sstart'])


In [ ]:
coll = pps.groupby('sseqid').apply(pd.DataFrame.sort_values, 'sstart').drop(['sseqid'], axis=1).reset_index()
collision_index = list(coll[(coll.sstart.diff() < 5) & (coll.sstart.diff() >= 0)].index)
collision_index.extend([i-1 for i in collision_index if i-1 not in collision_index])
collision_index.sort()
unique = coll[~coll.index.isin(collision_index)]
collisions = coll.iloc[collision_index]

def row_to_barcode(structure, row, ):
    bc = Barcode(structure)
    bc.bc_seq = r.barcode
    bc.chr = r.sseqid
    bc.sstart = r.sstart
    bc.send = r.send
    bc.strand = r.sstrand
    bc.multimap = r.multimap
    bc.count = r.total_count
    return bc


bcs = []
final_bcs = []
for i, r in collisions.iterrows():
    bcs.append(row_to_barcode('GTGTATAAGAGACAG:17:13:before', r))

for i, r in unique.iterrows():
    final_bcs.append(row_to_barcode('GTGTATAAGAGACAG:17:13:before', r))
    
bc = bcs.pop(0)
cnt = bc.count
while len(bcs) > 0:
    bc2 = bcs.pop(0)
    if bc.chr != bc2.chr or abs(bc.sstart - bc2.sstart) > 5:
        bc.count = cnt 
        final_bcs.append(bc)
        bc = bc2
        cnt = bc2.count
    else:
        if bc.editdistance(bc2) > 3:
            bc.count = cnt
    
            final_bcs.append(bc)
            bc = bc2
            cnt = bc2.count
        else:
            cnt += bc2.count
            if bc.count <  bc2.count:
                bc = bc2
ftest1 = pd.DataFrame([[bc.chr, bc.sstart, bc.bc_seq, bc.count] for bc in final_bcs], columns=['sseqid', 'sstart', 'barcode', 'total_count'])

In [ ]:
pps = df[['sseqid', 'sstart', 'sstrand', 'barcode', 'total_count', 'multimap']]
positions_sorted = (pps.groupby('sseqid')
        .apply(pd.DataFrame.sort_values, 'sstart')
        .drop(['sseqid'], axis=1)
        .reset_index()
        .drop(['level_1'], axis=1))

# Get indices for rows with collisions
collision_index = list(positions_sorted[(positions_sorted.sstart.diff() < 5) & (positions_sorted.sstart.diff() >= 0)].index)
collision_index.extend([i-1 for i in collision_index if i-1 not in collision_index])
collision_index.sort()

# Barcodes without collisions

unique = positions_sorted[~positions_sorted.index.isin(collision_index)]

collisions = positions_sorted.iloc[collision_index]

def row_to_barcode(structure, row):
    bc = Barcode(structure)
    bc.bc_seq = row.barcode
    bc.chr = row.sseqid
    bc.sstart = row.sstart
    bc.strand = row.sstrand
    bc.multimap = row.multimap
    bc.count = row.total_count
    return bc


bcs = []
final_bcs = []
for i, r in collisions.iterrows():
    bcs.append(row_to_barcode('GTGTATAAGAGACAG:17:13:before', r))
    
bc = bcs.pop(0)
cnt = bc.count
while len(bcs) > 0:
    bc2 = bcs.pop(0)
    if bc.chr != bc2.chr or abs(bc.sstart - bc2.sstart) > 5:
        bc.count = cnt 
        final_bcs.append(bc)
        bc = bc2
        cnt = bc2.count
    else:
        if bc.editdistance(bc2) > 3:
            bc.count = cnt
            final_bcs.append(bc)
            bc = bc2
            cnt = bc2.count
        else:
            cnt += bc2.count
            if bc.count <  bc2.count:
                bc = bc2
if bc not in final_bcs:
    final_bcs.append(bc)
resolved_collisions = (pd.DataFrame([[bc.chr, bc.sstart, bc.strand,  bc.bc_seq, bc.count, bc.multimap] for bc in final_bcs], 
                                    columns=['sseqid', 'sstart', 'sstrand', 'barcode', 'total_count', 'multimap']))

ftest = pd.concat([ unique, resolved_collisions])

In [ ]:
ftest[['barcode', 'total_count', 'sstart',
       'sseqid', 'sstrand', 'multimap']].to_csv("/Users/ansintsova/git_repos/mbarq/tests/"\
                                                "expected_outcomes/mapping/merge_colliding_bcs_new.csv")

In [ ]:
old_resolved


In [ ]:
old_resolved = pd.read_csv("/Users/ansintsova/git_repos/mbarq/tests/expected_outcomes/mapping/merge_colliding_bcs.csv")

In [ ]:
old_resolved

In [ ]:
ftest

In [ ]:
old_resolved.columns = [f'old_{c}' if c != 'barcode' else c for c in old_resolved.columns ]

In [ ]:
test = ftest.merge(old_resolved, how='outer', on='barcode')

In [ ]:
with_collisions = list(pps.barcode.values)
without_collisions = list(ftest.barcode.values)

In [ ]:
def editdist(x,y):
    dist = 0
    for letter1, letter2 in zip(x, y):
        if letter1 != letter2:
            dist += 1
    return dist

In [ ]:
import random
edits = []
edits2 = []
for i in range(100000):
    x, y = random.sample(with_collisions, 2)
    m,n = random.sample(without_collisions, 2)
    edits.append(editdist(x, y))
    edits2.append(editdist(m, n))
pd.Series(edits).hist(bins=100)
pd.Series(edits2).hist(bins=100)
plt.xlim(0,5.5 )
plt.ylim(0, 40)

In [ ]:
plt.plot(test.total_count, test.old_cnt, '.k')
plt.xlim(0, 25)
plt.ylim(0,25)

In [ ]:
final = pd.concat([unique, col_resolved])

In [ ]:
y = final.merge(res, how='outer', on='barcode')[['sseqid_x', 'sseqid_y', 'sstart_x', 'sstart_y', 'barcode', 'total_count', 'cnt_y']]

In [ ]:
y[y.sstart_y.isna()]

In [ ]:
coll.iloc[[424, 425]]

In [ ]:
coll[coll.sstart == 67688]

In [ ]:
coll[coll.barcode == 'CATAAATCCTTTGCACC']

In [ ]:
from typing import Optional

In [ ]:
class InputFileError(Exception):
    pass


class FastA(object):
    '''
    Standard data container for fasta sequences
    '''
    __slots__ = ['header', 'sequence']

    def __init__(self, header: str, sequence: str) -> None:
        self.header = header
        self.sequence = sequence


class Barcode:
    def __init__(self, structure):
        self.structure: str = structure
        self.bc_seq: Optional[str] = None
        self.host: Optional[str] = None
        self.bc_len: int
        self.tn_seq: str
        self.count: int = 0
        self.tn_before_bc: bool
        self.len_spacer: int
        # In theory these are optional
        self.host: str
        self.start: int
        self.end: int
        self.chr: str
        self.identifiers: List[str]
        self.eval: float
        self.bitscore: float
        self.alignment_len: int
        self._parse_structure()

    def _parse_structure(self):
        self.tn_seq = self.structure.split(':')[0]
        self.len_spacer = int(self.structure.split(':')[2])
        self.bc_len = int(self.structure.split(':')[1])
        self.tn_before_bc = True if self.structure.split(':')[3] == 'before' else False

    def editdistance(self, other_barcode) -> int:
        '''
        Calculate the edit distance between 2 sequences with identical length.
        Will throw an error if the length of both sequences differs
        :param seq1:
        :param seq2:
        :return:
        '''
        if not self.bc_seq or not other_barcode.bc_seq:
            raise Exception("Could not find sequence for one or more barcodes")
        if self.bc_seq == other_barcode.bc_seq:
            return 0
        if len(self.bc_seq) != len(other_barcode.bc_seq):
            raise Exception(
                f'{self.bc_seq} and {other_barcode.bc_seq} have different length. Edit distance can be computed on same length sequences only.')
        dist = 0
        for letter1, letter2 in zip(self.bc_seq, other_barcode.bc_seq):
            if letter1 != letter2:
                dist += 1
        return dist

    def extract_barcode_host(self, r1: FastA) -> None:
        '''
       Extract barcode and host sequence from read with tp2.
       Return (None, None) if the barcode sequence is not complete (17bp)
        :param r1:
        :return:

        -----|BARCODE|----------|TN end sequence (tp2)|---Host------
        -----|-bc_len-|--len_spacer--|---------tn_seq---------|-------------
        -----|-17bp--|---13bp---|---------15bp--------|----?--------
         ---(-30)---(-13)-------(0)---------------------------------
        '''

        splits: List[str] = r1.sequence.split(self.tn_seq)  # check that tn in sequence?
        if self.tn_before_bc:
            bc_start = -(self.bc_len + self.len_spacer)
            bc_end = -self.len_spacer
            bc_seq = splits[0]
            host_seq = splits[1]
        else:
            bc_start = self.len_spacer
            bc_end = self.len_spacer + self.bc_len
            bc_seq = splits[1]
            host_seq = splits[0]

        if len(bc_seq) >= self.len_spacer + self.bc_len:
            self.bc_seq = bc_seq[bc_start:bc_end]
            self.host = host_seq

In [ ]:
def merge_colliding_bcs(potential_positions: pd.DataFrame, edit_dist=3, logger= logging.getLogger()) -> pd.DataFrame:
    """

    Takes output of new_map_annotate, and merges colliding barcodes

    """
    pps = potential_positions[['sseqid', 'sstart', 'send', 'sstrand', 'barcode', 'cnt', 'total_count', 'multimap']]
    # Find barcodes mapped to the same position
    collisions = (potential_positions.groupby(['sseqid', 'sstart']).barcode.nunique()
                  .reset_index().rename({'barcode': 'num_uniq_bc'}, axis=1))
    # Get all the unique barcodes mapped to unique positions
    final_bcs = list(collisions[collisions.num_uniq_bc == 1].merge(pps, on=['sseqid', 'sstart']).barcode.values)
    # Collisions are only positions with more than one bc mapped
    collisions = collisions[collisions.num_uniq_bc > 1]
    collisions = collisions.merge(pps, on=['sseqid', 'sstart'])
    logger.info(f'Number of collissions: {collisions.shape[0]}')

    for position, df in collisions.groupby(['sseqid', 'sstart']):  # todo refactor
        # Get the barcode that was seen the most
        bc_idx = df.total_count.astype(int).idxmax()
        # That is likely the correct barcode
        likely_barcode = df.loc[bc_idx].barcode
        all_barcodes = list(df.barcode.values)
        all_barcodes.remove(likely_barcode)
        final_bcs.append(likely_barcode)

        # Check if edit distance of all other barcodes is > than specified, if yes, add it to the list as well
        # (or should ignore it? Different list? is this where other library contamination comes in?)
        for bc in all_barcodes:
            ced = editdistance(bc, likely_barcode)
            if ced > edit_dist:
                final_bcs.append(bc)

    barcode_map = pps[pps.barcode.isin(final_bcs)][
        ['barcode', 'cnt', 'sstart', 'send', 'sseqid', 'sstrand', 'multimap']]
    barcode_map = barcode_map.set_index('barcode')
    return barcode_map

In [ ]:
import logging

In [ ]:
res = merge_colliding_bcs(df)

In [ ]:
res.shape

In [ ]:
def editdistance(seq1: str, seq2: str) -> int:
    '''
    Calculate the edit distance between 2 sequences with identical length.
    Will throw an error if the length of both sequences differs
    :param seq1:
    :param seq2:
    :return:
    '''

    if seq1 == seq2:
        return 0
    if len(seq1) != len(seq2):
        raise Exception(f'{seq1} and {seq2} have different length. Edit distance can be computed on same length sequences only.')
    dist = 0
    for letter1, letter2 in zip(seq1, seq2):
        if letter1 != letter2:
            dist += 1
    return dist